In [1]:
import time
from math import floor
from rubik_cubes import Rubik, rubik_corner, rubik_edge
from rubik_state import rubik_state
from rubik_next import get_nexts_1, get_nexts_2
from rubik_file import RubikFile
from rubik_ida import IDA
from itertools import permutations
import copy

In [2]:
rubik = Rubik()

In [3]:
def new_get_nexts_1(state, h):
    return [rubik_state(state, i, h) for i in range(1, 19)]

In [30]:
def brute_force_co(save, solved):
    current = solved
    save[current.corners_state] = current.g
    queue = [current]
    
    while len(queue):
        current = queue.pop(0)
        for elem in get_nexts_1(current, null_h):
            if elem.corners_state not in save.keys():
                save[elem.corners_state] = elem.g
                queue.append(elem)

def brute_force_eo(save, solved):
    current = solved
    save[current.edges_state] = current.g
    queue = [current]
    
    while len(queue):
        current = queue.pop(0)
        for elem in get_nexts_1(current, null_h):
            if elem.edges_state not in save.keys():
                save[elem.edges_state] = elem.g
                queue.append(elem)

def brute_force_slice(save, solved):
    current = solved
    perm = permutations([5, 6, 7, 8])
    queue = [current]
    for elem in perm:
        new_dep = copy.deepcopy(solved)
        for i in range(5, 9):
            new_dep.corners[i].position = elem[i - 5]
        queue.append(new_dep)
        new_dep.slice_state = elem
        save[new_dep.slice_state] = 0
    save[current.slice_state] = current.g
    set1 = set()
    set2 = set()
    set3 = set()
    set4 = set()
    while len(queue):
        current = queue.pop(0)
        for elem in new_get_nexts_1(current, null_h):
            if elem.slice_state not in save.keys():
                save[elem.slice_state] = elem.g
                queue.append(elem)
                set1.add(tuple((elem.edges[1].final_position, elem.edges[9].final_position)))
                set2.add(tuple((elem.edges[2].final_position, elem.edges[10].final_position)))
                set3.add(tuple((elem.edges[3].final_position, elem.edges[11].final_position)))
                set4.add(tuple((elem.edges[4].final_position, elem.edges[12].final_position)))
            else:
                if save[elem.slice_state] > elem.g:
                    save[elem.slice_state] = elem.g
                if tuple((elem.edges[1].final_position, elem.edges[9].final_position)) not in set1:
                    queue.append(elem)
                    set1.add(tuple((elem.edges[1].final_position, elem.edges[9].final_position)))
                elif tuple((elem.edges[2].final_position, elem.edges[10].final_position)) not in set2:
                    queue.append(elem)
                    set2.add(tuple((elem.edges[2].final_position, elem.edges[10].final_position)))
                elif tuple((elem.edges[3].final_position, elem.edges[11].final_position)) not in set3:
                    queue.append(elem)
                    set3.add(tuple((elem.edges[3].final_position, elem.edges[11].final_position)))
                elif tuple((elem.edges[4].final_position, elem.edges[12].final_position)) not in set4:
                    queue.append(elem)
                    set4.add(tuple((elem.edges[4].final_position, elem.edges[12].final_position)))

def null_h(obj):
    return 0

In [31]:
solved = rubik_state(rubik, 0, null_h)
save_slice = {}
brute_force_slice(save_slice, solved)

In [32]:
tot = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for k, v in save_slice.items():
    tot[v] += 1
for i in range(0, len(tot)):
    print("nb_moves :", i, "nb_occ :", tot[i])
print(len(save_slice))

nb_moves : 0 nb_occ : 24
nb_moves : 1 nb_occ : 8
nb_moves : 2 nb_occ : 128
nb_moves : 3 nb_occ : 1248
nb_moves : 4 nb_occ : 4806
nb_moves : 5 nb_occ : 4850
nb_moves : 6 nb_occ : 737
nb_moves : 7 nb_occ : 70
nb_moves : 8 nb_occ : 9
nb_moves : 9 nb_occ : 0
nb_moves : 10 nb_occ : 0
nb_moves : 11 nb_occ : 0
11880


In [21]:
# print(sum([i for i in save_slice.values()]) / len(save_slice))
# print(len(save_slice))
tot = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for k, v in save_slice.items():
    tot[v] += 1
for i in range(0, len(tot)):
    print("nb_moves :", i, "nb_occ :", tot[i])

nb_moves : 0 nb_occ : 24
nb_moves : 1 nb_occ : 8
nb_moves : 2 nb_occ : 128
nb_moves : 3 nb_occ : 1248
nb_moves : 4 nb_occ : 4806
nb_moves : 5 nb_occ : 4850
nb_moves : 6 nb_occ : 737
nb_moves : 7 nb_occ : 70
nb_moves : 8 nb_occ : 9
nb_moves : 9 nb_occ : 0
nb_moves : 10 nb_occ : 0
nb_moves : 11 nb_occ : 0


In [6]:
solved = rubik_state(rubik, 0, null_h)
save_co = {}
brute_force_co(save_co, solved)

In [7]:
solved = rubik_state(rubik, 0, null_h)
save_eo = {}
brute_force_eo(save_eo, solved)

In [8]:
print(sum([i for i in save_slice.values()]) / len(save_slice))
print(len(save_slice))
print(sum([i for i in save_co.values()]) / len(save_co))
print(len(save_co))
print(sum([i for i in save_eo.values()]) / len(save_eo))
print(len(save_eo))

4.416329966329966
11880
4.459076360310928
2187
4.60888671875
2048


In [9]:
rubikfile = RubikFile("pruning_phase_1.h5")

rubikfile.write("corners", save_co)
rubikfile.write("edges", save_eo)
rubikfile.write("slice", save_slice)

Writing...
Write done
Writing...
Write done
Writing...
Write done


In [2]:
# phase two generator, to be reworked
def compress_state(state):
    compressed = 0
    # first compress corners place
    for i in range(1, 9):
        compressed <<= 3
        compressed |= (state.corners[i].final_position - 1)
    # compress corners orientation
    for i in range(1, 9):
        compressed <<= 2
        compressed |= state.corners[i].orientation
    # compress edges place
    for i in range(1, 13):
        compressed <<= 4
        compressed |= state.edges[i].final_position
    # corners edges orientation
    for i in range(1, 13):
        compressed <<= 1
        compressed |= state.edges[i].orientation
    return str(compressed)

def gen_pruning(dico, curr_node, depth, h):
    if depth < 9:
        next_depth = []
        for elem in get_nexts_2(curr_node, h):
            cp = compress_state(elem)
            if cp not in dico.keys():
                dico[cp] = depth + 1
                next_depth.append(elem)
            elif dico[cp] > depth + 1:
                dico[cp] = depth + 1
        for elem in next_depth:
            gen_pruning(dico, elem, depth + 1, h)

dico = {}
rubik = Rubik()
dep = rubik_state(rubik, 0, rubik.heuristic_h2)
dico[compress_state(dep)] = 0

start = time.time()
gen_pruning(dico, dep, 0, rubik.heuristic_h2)
print(time.time() - start)
print(len(dico))

In [32]:
tuple(5, 4)

TypeError: tuple expected at most 1 arguments, got 2